# Tool Wrappers

> Automatically generate DSPy modules from our component registry.

This module takes the component registry and generates fully-typed DSPy modules for each tool. It converts the component metadata into actual DSPy Module classes with proper signatures and documentation.

In [ ]:
#| default_exp wrappers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import re
import inspect
import dspy
from cogitarelink_dspy.components import COMPONENTS

In [ ]:
#| export
def parse_signature(sig_str):
    """Parse a signature string like 'foo(a:str, b:int) -> str' into parameter names,
    types, and return type.
    
    Args:
        sig_str: A string in the format "function_name(param1:type, param2:type) -> return_type"
                 or just "param1:type, param2:type) -> return_type"
                 
    Returns:
        tuple: (list of (name, type) tuples for parameters, return_type)
    """
    # Extract the part inside parentheses if it's a full function signature
    if '(' in sig_str:
        params_str = sig_str.split('(')[1].split(')')[0]
    else:
        params_str = sig_str
        
    # Extract return type if present
    return_type = None
    if ' -> ' in sig_str:
        return_type = sig_str.split(' -> ')[1].strip()
        
    # Parse parameters
    params = []
    if params_str.strip():
        for param in params_str.split(','):
            param = param.strip()
            if ':' in param:
                name, type_hint = param.split(':')
                params.append((name.strip(), type_hint.strip()))
            else:
                # If no type hint, default to str
                params.append((param.strip(), 'str'))
    
    return params, return_type

In [ ]:
# Test the signature parser
test_signatures = [
    ("forward(message:str)", [(("message", "str"))], None),
    ("load(source:str) -> dict", [(("source", "str"))], "dict"),
    ("validate(subject:str, predicate:str, object:str)", 
     [("subject", "str"), ("predicate", "str"), ("object", "str")], None),
    ("verify(graph_id:str, signature:str) -> bool", 
     [("graph_id", "str"), ("signature", "str")], "bool")
]

for sig_str, expected_params, expected_return in test_signatures:
    params, return_type = parse_signature(sig_str)
    assert params == expected_params, f"For {sig_str}, expected {expected_params}, got {params}"
    if expected_return:
        assert return_type == expected_return, f"For {sig_str}, expected return type {expected_return}, got {return_type}"
    
print("All signature parser tests passed!")

In [ ]:
#| export
def make_tool_wrappers(registry=COMPONENTS):
    """Generate DSPy Module classes for each tool in the registry.
    
    Args:
        registry: Dictionary of component definitions with layer, tool, doc, and calls fields
                 
    Returns:
        list: A list of DSPy Module classes, one for each component
    """
    tools = []
    
    for name, meta in registry.items():
        # Get call signature from the component metadata
        call_sig = meta["calls"]
        params, return_type = parse_signature(call_sig)
        
        # Create signature string for DSPy
        # Add commas between parameters for DSPy's signature parser
        param_sig = ", ".join(f"{p[0]}:{p[1]}" for p in params)
        output_type = return_type if return_type else "output"
        signature_str = f"{param_sig} -> {output_type}"
        
        # Create a new DSPy Module class with documentation and layer info
        class_doc = f"{meta['doc']} [Layer: {meta['layer']}]"
        
        class ToolWrapper(dspy.Module):
            """Placeholder docstring that will be replaced."""
            signature = dspy.Signature(signature_str)
            
            def forward(self, **kwargs):
                # This is just a stub - would actually call real implementation
                print(f"Called {meta['tool']} with args: {kwargs}")
                return f"Result from {meta['tool']}"
        
        # Set proper class name and docstring
        ToolWrapper.__doc__ = class_doc
        ToolWrapper.__name__ = meta['tool']
        ToolWrapper.__qualname__ = meta['tool']
        
        tools.append(ToolWrapper)
    
    return tools

In [ ]:
#| export
# Delay tool generation until explicitly called
TOOLS = None

def get_tools():
    """Get or initialize the tool wrappers.
    
    Returns:
        list: A list of DSPy Module classes, one for each component
    """
    global TOOLS
    if TOOLS is None:
        TOOLS = make_tool_wrappers()
    return TOOLS

# Helper function to organize tools by layer
def group_tools_by_layer(tools=None):
    """Group the generated tools by their semantic layer.
    
    Args:
        tools: List of tool classes to group. If None, uses get_tools().
        
    Returns:
        dict: A dictionary with layers as keys and lists of tools as values
    """
    if tools is None:
        tools = get_tools()
        
    result = {}
    for tool in tools:
        # Extract layer from the docstring
        doc = tool.__doc__
        layer_match = re.search(r'\[Layer: ([^\]]+)\]', doc)
        if layer_match:
            layer = layer_match.group(1)
            if layer not in result:
                result[layer] = []
            result[layer].append(tool)
    return result

In [ ]:
# Test the tool wrapper generation
tools = get_tools()
assert len(tools) == len(COMPONENTS), f"Expected {len(COMPONENTS)} tools, got {len(tools)}"

# Check that we have at least one tool for each layer
layers_dict = group_tools_by_layer(tools)
print(f"Generated tools by layer: {', '.join(layers_dict.keys())}")
for layer, tools in layers_dict.items():
    print(f"- {layer}: {len(tools)} tools")
    for tool in tools:
        print(f"  - {tool.__name__}")

# Test a specific tool
echo_tool = next(tool for tool in tools if tool.__name__ == "EchoMessage")
assert echo_tool.__name__ == "EchoMessage"
assert "Layer: Utility" in echo_tool.__doc__
assert "echoes the input message back" in echo_tool.__doc__.lower()

# Create and test an echo tool instance
echo_instance = echo_tool()
result = echo_instance(message="Hello, testing tool wrapper!")
print(f"\nTool result: {result}")

## Using the Generated Tools

Each tool is a DSPy Module class that can be instantiated and used in a DSPy pipeline. Here's how to use the tools:

1. **Individual Tool Usage**:
   - Instantiate a specific tool using its class
   - Call it with the appropriate parameters as defined in its signature

2. **Layer-Based Tool Selection**:
   - Use the `group_tools_by_layer()` function to organize tools by layer
   - Select tools from the appropriate layer based on the user's query
   
3. **Integration with DSPy Agent**:
   - Pass the entire `TOOLS` list to a `dspy.StructuredAgent`
   - The agent will be able to discover and use the tools based on their signatures and documentation

In [ ]:
# Example: Create an agent that can use our tools
def create_semantic_agent(lm=None):
    """Create a semantic agent with the generated tools.
    
    This function will eventually be moved to its own module. It's shown here
    as an illustration of how the tools will be used in a DSPy agent.
    """
    import dspy
    from cogitarelink_dspy.core import default_lm
    
    # Use the LLM from core if none is provided
    lm = lm or default_lm
    
    # Create a system prompt that explains the 4-layer architecture
    system_prompt = """
    You are a Semantic-Web agent that reasons over a 4-layer architecture:
    1. Context - Working with JSON-LD contexts and namespaces
    2. Ontology - Using vocabularies and ontology terms
    3. Rules - Applying validation rules and shapes
    4. Instances - Managing actual data instances
    5. Verification - Verifying and signing graph data
    
    Every tool is tagged with its PRIMARY layer. When answering a user question,
    pick the tool from the HIGHEST layer that suffices to answer the question.
    """
    
    semantic_lm = dspy.LM(lm.model, system=system_prompt) if lm else None
    
    # Create a StructuredAgent with our tools
    agent = dspy.StructuredAgent(
        tools=get_tools(),  # Get tools using the getter function
        lm=semantic_lm
    )
    
    return agent

# This will be implemented in a future notebook focused on the agent
# agent = create_semantic_agent() 
# result = agent.query("Load the schema.org context")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()